In [1]:
import psycopg2
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pyecharts
import urllib3,time,csv,datetime
import matplotlib as mpl
import matplotlib.pyplot as plt
# import matplotlib.finance as mpf
import matplotlib.dates as mpd
import plotly
import plotly.plotly as py
import plotly.offline as py_offline
import plotly.graph_objs as go
from IPython.display import clear_output

%matplotlib inline

ERROR:lml.utils:failed to import pyecharts_snapshot
Traceback (most recent call last):
  File "C:\Users\Brad Sun\Anaconda3\lib\site-packages\lml\utils.py", line 43, in do_import
    plugin_module = __import__(plugin_module_name)
ModuleNotFoundError: No module named 'pyecharts_snapshot'


# 项目总结

- 项目目标：结合张伟的系统，实时读取公司SQL数据库的数据，并画出资产蜡烛图，出现信号时提醒


- 完成进度：已经可以每隔一段时间，实时读取数据并分别展示出蜡烛图、一些技术指标与交易信号


- 未完成部分：将完成部分的功能连接到张伟数据库


- 未完成部分思路与建议：连接到系统的具体代码主要可以参考张伟系统quant -> plotter.py文件。可以创建一个大类，比如class Plot_CandleStick_Signals(object)，在这之中建立两个functions, 第一个def plot_candlestick(), 第二个def plot_signals().需要在def __init__(): 里准备的步骤有：1) 连接SQL数据库，并提取两个资产的OHLC数据来为plot_candlestick做准备；2)计算相对差以及相对差快慢均线，并定位信号出现的时点，为plot_signals()做准备。具体这两个准备步骤的代码基本已经写好并展示在下面的notebook内容里 

# 展示资产蜡烛图与生成信号

In [2]:
# 第一步：定义所有所需函数：

# 直接从SQL数据库生成资产价格dataframe
def two_assts_from_sql(asset1, asset2, sql_limit_num):
    conn = psycopg2.connect(database="bitmexdata", user="postgres", password="tongKen123", host="128.199.97.202", port="5432")
    asset1 = """ SELECT time, open, high, low, close, volume from {} order by id desc limit {}""".format(asset1, sql_limit_num)
    asset2 = """ SELECT time, open, high, low, close, volume from {} order by id desc limit {}""".format(asset2, sql_limit_num)
    df1 = pd.read_sql(asset1, con=conn)
    df2 = pd.read_sql(asset2, con=conn)
    conn.close()
    return df1, df2

# 转变数据周期：T/5T/15T/30T/H/2H/4H/D/W/M
def resample(df, period):
    convrted_df = df.resample(period).last()
    convrted_df['open'] = df['open'].resample(period).first()
    convrted_df['high'] = df['high'].resample(period).max()
    convrted_df['low'] = df['low'].resample(period).min()
    convrted_df['close'] = df['close'].resample(period).last()
    convrted_df['volume'] = df['volume'].resample(period).sum()
    # Keep rows with at least 5 non-NaN values
    convrted_df.dropna(thresh=5, inplace=True)
    convrted_df.index = convrted_df['time']
    convrted_df['time'] = pd.DatetimeIndex(convrted_df['time'])
    return convrted_df

# 为能够plot蜡烛图做timestamps处理的准备：
def cnvrt_date(convrted_df):
    cnvrted_date_df = convrted_df.copy()
    cnvrted_date_df['date'] = mpd.date2num(cnvrted_date_df['time'].dt.to_pydatetime())
    return cnvrted_date_df

# 计算两个资产的相对差累计值：
def two_assets_tmsum(cnvrted_date_df1, cnvrted_date_df2, N4):
    cnvrted_date_df1['close_shifted'] = cnvrted_date_df1['close'].shift(1)
    cnvrted_date_df2['close_shifted'] = cnvrted_date_df2['close'].shift(1)
    T1 = cnvrted_date_df1['close'].diff()/cnvrted_date_df1['close_shifted']
    T2 = cnvrted_date_df2['close'].diff()/cnvrted_date_df2['close_shifted']
    TM = T1*(1-N4/100) - T2*(N4/100)
    tmsum_sr = TM.cumsum()
    return tmsum_sr
    
# 计算相对差的移动平均快慢线：
def MAs_of_tmsum(tmsum_sr, N1, N2):
    MA1 = tmsum_sr.ewm(span= N1).mean() #快线
    MA2 = tmsum_sr.ewm(span= N2).mean() #慢线   
    return MA1, MA2
    
# Plot TMSUM图线：
def plot_tmsum(tmsum_sr):
#     py_offline.init_notebook_mode()
    tmsum_df = pd.DataFrame(tmsum_sr, columns=['tmsum'])
    tmsum_df = go.Scatter(x = tmsum_df.index,
                          y = tmsum_df['tmsum'])
    data = [tmsum_df]
    fig = go.Figure(data=data)
    return py.iplot(fig, filename='TMSUM')
    
    
def plot_tmsum_MAs(MA1, MA2):
    # Here I didn't use offline's version of plotly, going forward  
    # will need to be consistent when moving to pycharm for plotting
    ma1_df = pd.DataFrame(MA1, columns=['MA1'], index = MA1.index)
    ma2_df = pd.DataFrame(MA2, columns=['MA2'], index = MA2.index)
    trace1 = go.Scatter(x = ma1_df.index, 
                        y = ma1_df['MA1'])
    trace2 = go.Scatter(x = ma2_df.index, 
                        y = ma2_df['MA2'])
    data = [trace1, trace2]
    fig = go.Figure(data=data)
    return py.iplot(fig, filename='plot_tmsum_MAs')
    
    
# Plot 资产蜡烛图：
def plot_candlestick(cnvrted_date_df):
    candle_df = go.Candlestick(x = cnvrted_date_df.index,
                               open = cnvrted_date_df['open'],
                               high = cnvrted_date_df['high'],
                               low  = cnvrted_date_df['low'],
                               close = cnvrted_date_df['close'])
    data = [candle_df]
    fig =go.Figure(data=data)
    return py.iplot(fig, filename='CandleStick')


# Build a function that plots charts of two moving averages with their crossover trading signals 
# Here we want to make sure that both MA1_sr and MA2_sr are: series with timestanmps as their indexes

def MA_crossover_plot_signals(MA1_sr, MA2_sr):
    # 构建一个由 MA1和 MA2构成的dataframe：
    ma_signal_df = pd.DataFrame(MA1_sr, columns=['MA1'], index= MA1_sr.index)
    ma_signal_df['MA2'] = MA2_sr
    # 用两者的差表示 MA1在位置上高于还是低于MA2，负值说明低于，正值说明高于：
    ma_signal_df['MA1_mns_MA2'] = ma_signal_df['MA1'] - ma_signal_df['MA2']
    # 将正负值同义转换为二元的 1或者 -1便于观察和处理
    ma_signal_df['signs'] = ma_signal_df['MA1_mns_MA2']*abs(1/(ma_signal_df['MA1_mns_MA2']))
    # 用当前值和前一个周期的值决定当前状态是金叉信号还是死叉信号
    ma_signal_df['pre_signs'] = ma_signal_df['signs'].shift(1)
    ma_signal_df['signals'] = ma_signal_df['signs'] - ma_signal_df['pre_signs']
    ma_signal_df['signals_alert'] = ma_signal_df['signals'].apply(lambda x: "金叉" if x==2 else "死叉" if x==-2 else "无信号")
    # 将所有出现信号的rows挑出来建立一个字典：
    ma_df_with_signals = ma_signal_df[ma_signal_df['signals_alert'] != "无信号"]['signals_alert']
    signal_dict = dict(ma_df_with_signals)
    # Add each and all signal information(dictionary format) to annotations(list format) 
    # so that it can be put into the go.Layout() function.
    annotations = []
    each_dict = {}
    for i, k in enumerate(signal_dict):
        each_dict['x'] = k
        each_dict['y'] = ma_signal_df['MA1'].loc[k]
        each_dict['text']=ma_signal_df['signals_alert'].loc[k]
        each_dict['showarrow']=True
        each_dict['arrowhead']=7
        each_dict['ax']=0
        each_dict['ay']=-40

        annotations.append(each_dict.copy())
    # 用plotly进行绘图，包括了之前处理好的annotations，作为显示信号的功能
    ma1_df = pd.DataFrame(tmsum_ma1, columns=['MA1'], index = tmsum_ma1.index)
    ma2_df = pd.DataFrame(tmsum_ma2, columns=['MA2'], index = tmsum_ma2.index)
    trace1 = go.Scatter(x = ma1_df.index, 
              y = ma1_df['MA1'])
    trace2 = go.Scatter(x = ma2_df.index, 
              y = ma2_df['MA2'])
    layout = go.Layout(
        showlegend=False,
        annotations=annotations
    )
    data = [trace1, trace2]
    fig = go.Figure(data=data, 
                    layout=layout)
    return py.iplot(fig, filename='plot_MAs_with_signals')
#     return plotly.offline.plot(fig, filename = 'test.html') # 如果要用html那么则会以网页的形式输出图像

In [3]:
# 第二步：确定参数：
N1 = 5
N2 = 15
N3 = 15
N4 = 15
period = '5T'

# 第三步：链接数据库，提取两个资产
bch_df, eth_df = two_assts_from_sql("bchz18_1m", "ethz18_1m", 100)

# 第四步：转变时间周期
bch_df.index = pd.DatetimeIndex(bch_df['time'])
eth_df.index = pd.DatetimeIndex(eth_df['time'])
res_bch = resample(bch_df, period)
res_eth = resample(eth_df, period)

# 第五步：处理时间column以此方便后面的作图
cnvrted_date_df_bch = cnvrt_date(res_bch)
cnvrted_date_df_eth = cnvrt_date(res_eth)

# 第六步：计算相对差累积和、相对差累积和的快均线、相对差累计和的慢均线
tmsum_sr = two_assets_tmsum(cnvrted_date_df_bch, cnvrted_date_df_eth, N4)
tmsum_ma1, tmsum_ma2 = MAs_of_tmsum(tmsum_sr, N1, N2) 

In [4]:
# 第七步：作图、显示生成信号
# 做出资产一的蜡烛图：
plot_candlestick(cnvrted_date_df_eth)

In [5]:
# 做出资产二的蜡烛图
plot_candlestick(cnvrted_date_df_bch)

In [6]:
# 做出相对差累积值的图像
plot_tmsum(tmsum_sr)

In [7]:
# 生成初步信号：金叉死叉。
MA_crossover_plot_signals(tmsum_ma1, tmsum_ma2)

# 如何实时读取数据并展示:

到此，所有的读取与图像展示皆为静态，如果要实时从SQL数据抓取数据并运行以上代码，可以每隔一段时间自动运行一次即可。

In [3]:
# 例子：
while True:
    
    print ("waiting...")
        # 第二步：确定参数：
    N1 = 5
    N2 = 15
    N3 = 15
    N4 = 15
    period = '5T'

    # 第三步：链接数据库，提取两个资产
    bch_df, eth_df = two_assts_from_sql("bchz18_1m", "ethz18_1m", 100)

    # 第四步：转变时间周期
    bch_df.index = pd.DatetimeIndex(bch_df['time'])
    eth_df.index = pd.DatetimeIndex(eth_df['time'])
    res_bch = resample(bch_df, period)
    res_eth = resample(eth_df, period)

    # 第五步：处理时间column以此方便后面的作图
    cnvrted_date_df_bch = cnvrt_date(res_bch)
    cnvrted_date_df_eth = cnvrt_date(res_eth)

    # 第六步：计算相对差累积和、相对差累积和的快均线、相对差累计和的慢均线
    tmsum_sr = two_assets_tmsum(cnvrted_date_df_bch, cnvrted_date_df_eth, N4)
    tmsum_ma1, tmsum_ma2 = MAs_of_tmsum(tmsum_sr, N1, N2) 
    
    MA_crossover_plot_signals(tmsum_ma1, tmsum_ma2)
    
    time.sleep(30) 

waiting...
waiting...
waiting...
waiting...


DatabaseError: Execution failed on sql:  SELECT time, open, high, low, close, volume from bchz18_1m order by id desc limit 100
server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.

unable to rollback

In [ ]:
# 例子：
while True:
        # 第二步：确定参数：
    N1 = 5
    N2 = 15
    N3 = 15
    N4 = 15
    period = '5T'

    # 第三步：链接数据库，提取两个资产
    bch_df, eth_df = two_assts_from_sql("bchz18_1m", "ethz18_1m", 100)

    # 第四步：转变时间周期
    bch_df.index = pd.DatetimeIndex(bch_df['time'])
    eth_df.index = pd.DatetimeIndex(eth_df['time'])
    res_bch = resample(bch_df, period)
    res_eth = resample(eth_df, period)

    # 第五步：处理时间column以此方便后面的作图
    cnvrted_date_df_bch = cnvrt_date(res_bch)
    cnvrted_date_df_eth = cnvrt_date(res_eth)

    # 第六步：计算相对差累积和、相对差累积和的快均线、相对差累计和的慢均线
    tmsum_sr = two_assets_tmsum(cnvrted_date_df_bch, cnvrted_date_df_eth, N4)
    tmsum_ma1, tmsum_ma2 = MAs_of_tmsum(tmsum_sr, N1, N2) 
    # 生成初步信号：金叉死叉。
    MA_crossover_plot_signals(tmsum_ma1, tmsum_ma2)
    time.sleep(60) 
    # 如果是在ipython notebook上展示，用clear_output可以清除之前plot出来的图像，然后会再plot更新后的图像
    clear_output()